# Person Detection

In [3]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob
import cv2
import coco
import utils
import model as modellib
import visualize
import pandas as pd
import re
import datetime
import csv
import random

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.abspath('D:\\_CV_PROJECT\\scrins')

## Configurations

Используем модель, предобученную на MS-COCO dataset. Конфигурация модели находится в ```CocoConfig``` классе ```coco.py```.


In [4]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
  #  DETECTION_MIN_CONFIDENCE = 0.3

config = InferenceConfig()
config.display()


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING     

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [4]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Date and Time 's parse

In [5]:
def date_parse(date):
    reout = re.search(r'(?P<year>[0-9]{4})[ -\/17]*(?P<month>[0-9]{2})[ -\/17]*(?P<day>[0-9]{2})', date)
    if reout!=None:
        return "{year}-{month}-{day}"\
                    .format(day = reout.group('day'),
                            month = reout.group('month'),
                            year = '20'+reout.group('year')[2:])
    else:
        return 'date'
    
def time_parse(time):
    reout = re.search(r'(?P<hour>[0-9]{2})[ :]*(?P<minute>[0-9]{2})[ :]*(?P<second>[0-9]{2})', time)
    if reout!=None:
        if reout.group('hour')[0]=='8':
            hour='0'+ reout.group('hour')[1]
        elif reout.group('hour')=='00':
            hour='08'
        elif reout.group('hour')=='18' or reout.group('hour')=='16':
            hour='10'
        else:
            hour=reout.group('hour')
        return "{hour}:{minute}:{second}"\
                    .format(hour = hour,
                            minute = reout.group('minute'),
                            second = reout.group('second'))
    else:
        return 'time'

## Date and Time detection

In [6]:
def datetime_detection(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im = cv2.threshold(im, 180, 255, cv2.THRESH_BINARY)[1]
    api = PyTessBaseAPI("D:\\tesseract\\Tesseract-OCR\\", lang='eng+rus', psm = PSM.SINGLE_LINE)
    api.SetVariable("tessedit_char_whitelist", "/:-012345789 ")
    api.SetVariable("matcher_bad_match_pad", "0.25")
    scale=2
    
    d_im=im[564:(564+16),580:(580+70)]
    d_im=cv2.resize(d_im, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    d_im = cv2.threshold(d_im, 180, 255, cv2.THRESH_BINARY)[1]
    api.SetImageBytes(d_im.tobytes(),d_im.shape[1], d_im.shape[0], 1, d_im.shape[1]*1)
    api.SetRectangle(0,0,d_im.shape[1],d_im.shape[0])
    api.Recognize()
    date=date_parse(api.GetUTF8Text().replace(' ',''))

    d_im=im[588:(588+18),595:(595+55)]
    d_im=cv2.resize(d_im, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    d_im = cv2.threshold(d_im, 180, 255, cv2.THRESH_BINARY)[1]
    api.SetImageBytes(d_im.tobytes(),d_im.shape[1], d_im.shape[0], 1, d_im.shape[1]*1)
    api.SetRectangle(0,0,d_im.shape[1],d_im.shape[0])
    api.Recognize()
    time=time_parse(api.GetUTF8Text().replace(' ',''))
    return date, time

In [7]:
def datetime_draw(img, date, time, x=20, y=50):
    color = (0,0,0)
    return cv2.putText(img, date+' '+time, (x,y), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=color, thickness=2)

## Detection function

In [8]:
def pedestrian_detection(file,display=True):
    print('------------------------------------------------------------')
    
    image = skimage.io.imread(file)

    r = model.detect([image], verbose=1)[0]
    persons=r['class_ids']==1

    date, time = datetime_detection(image)

    data={'image directory':file, 'date':date, 'time':time, 'number of persons':sum(persons),\
         'rois':' '.join([str(l) for l in r['rois'][persons]])}
    print(data['image directory'])
    img = visualize.display_instances(image, r['rois'][persons], r['masks'][:,:,persons], 
                                          r['class_ids'][persons], class_names, r['scores'][persons])
    if display:
        cv2.imshow('',img)
        k=cv2.waitKey(700)
        cv2.destroyAllWindows()

    cv2.imwrite('D:\\_CV_PROJECT\\out2\\'+file[-10:],img)
        
    return data

## New journal's creation

In [ ]:
journal_columns=('image directory','date','time','number of persons','rois')
with open(IMAGE_DIR+'\\journal.csv','w',newline='') as f:
                    writer=csv.DictWriter(f,journal_columns)
                    writer.writerow({'image directory':'image directory', 'date':'date',\
                                     'time':'time', 'number of persons':'number of persons','rois':'rois'})
journal=pd.DataFrame(columns=journal_columns)
journal.to_csv(IMAGE_DIR+'\\journal.csv')

## Detection as is

In [ ]:
a=datetime.datetime.now().replace(microsecond=0)
while True:
    try:
        if (datetime.datetime.now().replace(microsecond=0)-a).total_seconds()%10==0:
            journal_img_dir=[]
            with open(IMAGE_DIR+'\\journal.csv','r') as f:
                reader=csv.DictReader(f,journal_columns)
                for row in reader:
                    journal_img_dir.append(row['image directory'])
            break_flg=len(glob.glob(IMAGE_DIR+'\\start.txt'))!=0
            file_names = list(set(glob.glob(IMAGE_DIR+'\\*.jpg'))-set(journal_img_dir))

            if len(file_names)>0:
                for file in file_names:
                    data=pedestrian_detection(file,display=False)
                    with open(IMAGE_DIR+'\\journal.csv','a',newline='') as f:
                        writer=csv.DictWriter(f,journal_columns)
                        writer.writerow(data)

                print('Processed '+str(len(file_names))+' images')

            if break_flg:
                break
    except:
        continue


------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img068.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img000.jpg

*** No instances to display *** 

------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1

D:\_CV_PROJECT\pres scrins\img077.jpg

*** No instances to display *** 

------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img097.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img049.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img081.jpg

*** No instances to display *** 

------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img036.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img0

D:\_CV_PROJECT\pres scrins\img039.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img032.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img040.jpg

*** No instances to display *** 

------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000

D:\_CV_PROJECT\pres scrins\img113.jpg

*** No instances to display *** 

------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img027.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img003.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img031.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img046.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img085.jpg

*** No instances to display

D:\_CV_PROJECT\pres scrins\img112.jpg
------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img073.jpg

*** No instances to display *** 

------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1131.00000
D:\_CV_PROJECT\pres scrins\img001.jpg

*** No instances to display *** 

------------------------------------------------------------
Processing 1 images
image                    shape: (671, 1131, 3)     